# 循环神经网络

## 1. 数据准备

将训练语料转换为字典形式

In [1]:
%matplotlib inline
import zipfile
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time

加载数据集

In [2]:
def load_data_jay_lyrics(num_sample=10000):
    """
    加载周杰伦歌词数据集
    """
    # 读取数据集
    with open('../dataset/jaychou_lyrics.txt', encoding='utf-8') as f:
        corpus_chars = f.read()
    # 把换行符替换为空格
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[:num_sample]
    # 建立字符索引
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    # 将训练集中每个字符转换为索引
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    # 返回索引后的前num_sample个字符的文本，字符到索引的映射，索引到字符的映射，字符表大小
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [3]:
corpus_indices, char_to_idx, idx_to_char, vocab_size = load_data_jay_lyrics()

In [4]:
vocab_size

1027

In [5]:
len(corpus_indices)   # 10000

10000

In [6]:
char_to_idx['风']

75

In [7]:
idx_to_char[946]

'欢'

## 2. 采样

### 2.1 随机采样

下⾯的代码每次从数据⾥随机采样⼀个小批量。其中批量⼤小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。在随机采样中，每个样本是原始序列上任意截取的⼀段序列。相邻的两个随机小批量在原始序列上的位置不⼀定相毗邻。因此，我们⽆法⽤⼀个小批量最终时间步的隐藏状态来初始化下⼀个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。

In [8]:
def data_iter_random(corpus_indices, batch_size, num_steps):
    '''
    corpus_indices: 词典按先后次序的索引
    batch_size: 每个批次的样本容量
    num_steps: 每个样本的长度
    '''
    num_samples = (len(corpus_indices) - 1) // num_steps  # 可取的样本数量 = 总词汇//样本长度
    epoch_size = num_samples // batch_size  # 每批次样本数 = 总词汇数量 // (样本长度 * 样本数量)
    sample_indices = list(range(num_sumples))  # 样本的索引，注意和词索引区分开
    np.random.shuffle(sample_indices)  # 打乱样本初始索引的顺序
      
    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        batch_indices = sample_indices[i*batch_size: (i + 1)*batch_size ]  # 样本索引
        X = [corpus_indices[j*num_steps: (j + 1)*num_steps] for j in batch_indices]  # context
        Y = [corpus_indices[j*num_steps + 1: (j + 1)*num_steps + 1] for j in batch_indices]  # 预测目标
        yield torch.IntTensor(X), torch.IntTensor(Y)

In [9]:
my_seq = list(range(30))
i = 0
for X, Y in data_iter_random(my_seq, batch_size=3, num_steps=6):
    print(i)
    print('X:\n', X, '\nY:\n', Y)
    i += 1

0
X:
 tensor([[12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23],
        [ 6,  7,  8,  9, 10, 11]], dtype=torch.int32) 
Y:
 tensor([[13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24],
        [ 7,  8,  9, 10, 11, 12]], dtype=torch.int32)


### 2.2 相邻采样

除对原始序列做随机采样之外，我们还可以令相邻的两个随机小批量在原始序列上的位置相毗邻。这时候，我们就可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。这对实现循环神经网络造成了两方面影响：一方面， 在训练模型时，我们只需在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。 为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。

In [10]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps):
    '''
    corpus_indices: 词典按先后次序的索引
    batch_size: 每个批次的样本容量
    num_steps: 每个样本的长度
    每次输出为: (batch_size, num_steps)
    '''
    corpus_indices = np.array(corpus_indices)
    data_len = len(corpus_indices)  # 词典中单词个数
    batch_len = data_len // batch_size  # 可用小批量的数量
    indices = corpus_indices[0: batch_size*batch_len].reshape(batch_size, batch_len)  # 先取总量，再塑形，可能有最后几个单词没有取到
    epoch_size = (batch_len - 1) // num_steps  # 批量数量
    for i in range(epoch_size):
        X = indices[:, i * num_steps: (i + 1) * num_steps]
        Y = indices[:, i * num_steps + 1: (i + 1) * num_steps + 1]
        yield torch.IntTensor(X), torch.IntTensor(Y)

In [11]:
my_seq = list(range(100))
for X, Y in data_iter_consecutive(my_seq, batch_size=4, num_steps=6):
    print('X:\n', X.transpose(0, 1), '\nY:\n', Y.transpose(0, 1))  # num_steps, batch_size
    break

X:
 tensor([[ 0, 25, 50, 75],
        [ 1, 26, 51, 76],
        [ 2, 27, 52, 77],
        [ 3, 28, 53, 78],
        [ 4, 29, 54, 79],
        [ 5, 30, 55, 80]], dtype=torch.int32) 
Y:
 tensor([[ 1, 26, 51, 76],
        [ 2, 27, 52, 77],
        [ 3, 28, 53, 78],
        [ 4, 29, 54, 79],
        [ 5, 30, 55, 80],
        [ 6, 31, 56, 81]], dtype=torch.int32)


## 2. `one_hot`编码

> 对语料中的每个不同单词进行`one_hot`编码。编码长度为字典长度，单词的索引对应的位置的编码值为1，其余为0。

为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。假设词典中不同字符的数量为$N$（即词典大小`vocab_size`），每个字符已经同一个从0到$N-1$的连续整数值索引一一对应。如果一个字符的索引是整数$i$, 那么我们创建一个全0的长为$N$的向量，并将其位置为$i$的元素设成1。该向量就是对原字符的one-hot向量。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [12]:
def one_hot(word_indices, vocab_size):
    '''
    word_indices: 需要编码的索引, torch.IntTensor
    vocab_size: 词典大小, scalar
    '''
    shape = list(word_indices.shape) + [vocab_size]
    res = torch.zeros(size=shape)
    if len(shape) == 2:
        res[range(shape[0]), word_indices] = 1
    elif len(shape) == 3:
        for i in range(shape[0]):
            for j in range(shape[1]):
                res[i, j, word_indices[i, j]] = 1
    else:
        print('X超过2维!')

    return res

In [13]:
x = torch.IntTensor([[1, 2],[2, 3],[3, 4]])
one_hot(x, 10)

tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]])

每次采样的小批量的形状是(批量大小, 时间步数)，以下函数将字符下标转换成字符的`one-hot`编码。

In [14]:
def to_onehot(batch_step, size):
    """
    X: n*t, n为批量大小，t为时间步长
    size: 词典大小
    返回包含t个矩阵的列表, x: (批量大小, 词典大小)
    """
    return [one_hot(x, size) for x in batch_step.t()]  # 注意转置: (step, batch, vocab_size)

In [15]:
batch_step = torch.arange(20).reshape(4, 5)
inputs = to_onehot(batch_step, 20)

In [49]:
torch.arange(20).reshape(4, 5)

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])

In [16]:
batch_step.t()

tensor([[ 0,  5, 10, 15],
        [ 1,  6, 11, 16],
        [ 2,  7, 12, 17],
        [ 3,  8, 13, 18],
        [ 4,  9, 14, 19]])

In [17]:
len(inputs), inputs[0].shape  # 时间步, (小批量大小, 输入数量)

(5, torch.Size([4, 20]))

In [18]:
torch.stack(inputs)

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         

## 3. `RNN`模型从零实现

RNN模型的实现分为3部分：模型参数的初始化、隐藏层的初始化和模型主体部分

### 3.1 初始化模型参数

In [19]:
def get_rnn_params(num_inputs, num_hiddens, num_outputs):
    '''
    三层
    num_inputs: 输入层结点数量
    num_hiddens: 隐藏层结点数量
    num_outputs: 输出层结点数量
    '''
    # 隐藏层参数
    W_xh = torch.randn(num_inputs, num_hiddens) * 0.01
    W_hh = torch.randn(num_hiddens, num_hiddens) * 0.01
    b_h = torch.zeros(num_hiddens)

    # 输出层参数
    W_ho = torch.randn(num_hiddens, num_outputs) * 0.01
    b_o = torch.zeros(num_outputs)

    # 附上梯度
    W_xh.requires_grad_(True)
    W_hh.requires_grad_(True)
    b_h.requires_grad_(True)
    W_ho.requires_grad_(True)
    b_o.requires_grad_(True)
    return W_xh, W_hh, b_h, W_ho, b_o

### 3.2 初始化隐藏层的值

隐藏层的初始化值需要给第一个时间步的隐藏层使用

In [20]:
def init_rnn_hidden_state(batch_size, num_hiddens):
    '''
    batch_size: 每个批量的样本量
    num_hiddens: 隐藏层结点数量
    '''
    return torch.zeros(batch_size, num_hiddens)

### 3.3 构建循环神经网络

In [21]:
def rnn(inputs, state, params):
    '''
    inputs: 各时间步对应(batch_size, vocab_size)构成的张量
    state: 初始隐藏层结点状态(batch_size, num_hiddens)
    params: 输入(t)->隐藏(t)，隐藏(t-1)->隐藏(t)，隐藏(t)->输出(t+1) 参数
    计算len(inputs)时间步， 初始状态为state，以及参数为params下的小批量样本对应的输出
    注意：该模型的输入层的神经元数量为字典大小，即对应一个单词
    '''
    W_xh, W_hh, b_h, W_ho, b_o = params
    H = state
    outputs = []
    for X in inputs:  # X: batch_size * vocab_size 批次大小*词典大小
        H = torch.tanh(X@W_xh + H@W_hh + b_h)  # batch_size * hidden_size
        Y = H@W_ho + b_o  # batch_size * vocab_size
        outputs.append(Y)

    return outputs, H  # 相邻采样时, H作为下一个批量的初始状态

In [22]:
X = torch.arange(20).reshape(4, 5)  # 一个批次，样本量为4，时间步为5
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size  # 输入层数量，隐藏层数量，输出层数量
state = init_rnn_hidden_state(X.shape[0], num_hiddens) # 初始化隐藏层神经元的输出
inputs = to_onehot(X, vocab_size)  # 将由索引表示的词转换为one hot向量
params = get_rnn_params(num_inputs, num_hiddens, num_outputs)  # 初始化模型参数
outputs, state_new = rnn(inputs, state, params)  # 模型输出

In [23]:
X.shape

torch.Size([4, 5])

In [24]:
len(outputs), outputs[0].shape, state_new[0].shape

(5, torch.Size([4, 1027]), torch.Size([256]))

### 3.4 预测前n个字符

In [25]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_hidden_state, num_hiddens, vocab_size, idx_to_char, char_to_idx):
    """
    prefix: 初始单词
    num_chars: 往前预测的单词数量
    rnn: rnn模型
    params: 输入rnn模型的参数
    init_rnn_state: 初始化模型参数
    num_hiddens: 隐藏层结点数量
    vocab_size: 字典中的单词个数
    idx_to_char: {索引: 单词}
    char_to_idx: {字符: 索引}
    """
    state = init_rnn_hidden_state(1, num_hiddens)  # 1对应prefix的纬度
    output = [char_to_idx[c] for c in prefix]  # 初始化输出为prefix，得到对单词应的索引
    for t in range(num_chars - 1):
        # one-hot向量占据很大的内存，所以借由每批次单独处理减少内存的使用
        # output[-1]为最近预测的一个单词
        idx_X = torch.LongTensor([[output[-1]]])
        # 转换为one hot向量
        X = to_onehot(idx_X, vocab_size)
        # 预测输出和隐藏层的当前状态（用于下一次预测）
        Y, state = rnn(X, state, params)
        # 输出为对应的单词索引
        output.append(int(Y[0].argmax(dim=1).item()))
            
    return ''.join([idx_to_char[i] for i in output])

In [26]:
params = get_rnn_params(num_inputs, num_hiddens, num_outputs)
prefix = '回家去吧'
predict_rnn(prefix, 30, rnn, params, init_rnn_hidden_state, num_hiddens, vocab_size, idx_to_char, char_to_idx)

'回家去吧梦彿课处物谁长知它胸翻甩银性盘彿课处物谁长知它胸翻甩银性盘'

### 3.5 裁减梯度

循环神经网络中较容易出现梯度衰减或梯度爆炸。为了应对梯度爆炸，我们可以裁剪梯度（`clip gradient`）。假设我们把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度

$$ \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}$$

的$L_2$范数不超过$\theta$。

In [27]:
def grad_clipping(params, theta):
    norm = torch.tensor(0.0)
    for param in params: # 计算范
        norm += torch.norm(param.grad, 2)
        
    if norm > theta:
        for param in params:
            param.grad.data.mul_(theta / norm)

In [28]:
torch.norm(torch.FloatTensor([1,2,3]), 2)

tensor(3.7417)

**模型困惑度**

我们通常使用困惑度（perplexity）来评价语言模型的好坏。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`。

### 3.6 训练模型

In [29]:
def train_and_predict_rnn(rnn, get_rnn_params, init_rnn_state, num_hiddens, 
                          corpus_indices, vocab_size, idx_to_char, 
                          char_to_idx, is_random_iter, num_epochs, 
                          num_steps, lr, clipping_theta, batch_size, 
                          prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
        
    params = get_rnn_params(vocab_size, num_hiddens, vocab_size)  # 获取模型初始参数值
    loss =  nn.CrossEntropyLoss()  # 定义交叉熵损失函数
    start = time.perf_counter()
    for epoch in range(num_epochs):
        if not is_random_iter:  # 初始化相邻抽样的隐藏层状态
            state = init_rnn_state(batch_size, num_hiddens)
            
        l_sum, n = 0.0, 0
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps)  # 抽样字符的下标
        for X, Y in data_iter:  # 依次取出小批量
            if is_random_iter:  # 如果是随机采样，每个小批量初始化隐藏层状态
                state = init_rnn_state(batch_size, num_hiddens)
            else:  # 如果是相邻采样，则每个小批量取上一个训练最后时刻的隐藏层状态，但s需要从上一个周期计算图中解耦
                for s in state:
                    s.detach_()  # 从上一周期计算图解耦
            
            inputs = to_onehot(X.long(), vocab_size)  # 转换成one-hot向量
            outputs, state = rnn(inputs, state, params)  # outputs是num_steps个形状为(batch_size, len(vocab))的矩阵
            outputs = torch.cat(outputs, dim=0)  # 将num_steps个矩阵合并成一个矩阵(num_steps * batch_size, len(vocab))
            y = Y.t().reshape((-1,))  # Y的原形状为(batch_size, num_steps), 将其转换为和output一致的数组y
            l = loss(outputs, y.long()).mean()  # 通过交叉熵度量分类错误
            l.backward()  # 反向传播，自动计算梯度
            with torch.no_grad():
                grad_clipping(params, clipping_theta)  # 裁剪梯度
                for param in params:  # 小批量梯度下降
                    param.data.sub_(lr*param.grad/batch_size)
                    param.grad.data.zero_()
                
            l_sum += l.item() * y.numel()  # 总损失
            n += y.numel()  # 已训练样本数量
            
        if (epoch + 1) % 50 == 0:  # 每50批次运行一次
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, np.exp(l_sum / n), time.perf_counter() - start))
            start = time.perf_counter()  # 重置开始时间
            
        if (epoch + 1) % 100 == 0:  # 预测未来的50个字符
            for prefix in prefixes:
                print(' -',  predict_rnn(prefix, 50, rnn, params, init_rnn_hidden_state, num_hiddens, vocab_size, idx_to_char, char_to_idx))

### 3.7 训练模型并创作歌词

现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀“分开”和“不分开”分别创作长度为50个字符（不考虑前缀长度）的一段歌词。我们每过50个迭代周期便根据当前训练的模型创作一段歌词。

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 100, 35, 32, 0.05, 1e-2
pred_period, pred_len, prefixes = 20, 20, ['分开', '不分开']

下面采用随机采样训练模型并创作歌词。

In [32]:
train_and_predict_rnn(rnn, get_rnn_params, init_rnn_hidden_state, 
                      num_hiddens, corpus_indices, vocab_size,
                      idx_to_char, char_to_idx, True, 
                      num_epochs, num_steps, lr, 
                      clipping_theta, batch_size, prefixes)

epoch 50, perplexity 1026.244786, time 16.76 sec
epoch 100, perplexity 1025.473185, time 16.81 sec
 - 分开                                                 
 - 不分开                                                 


In [33]:
predict_rnn('分开', 50, rnn, params, init_rnn_hidden_state, num_hiddens, vocab_size, idx_to_char, char_to_idx)

'分开明奔近和加横墓拖o壁带山毫呜爵隐密睛属茶翔己但试重专也怕属茶翔己但试重专也怕属茶翔己但试重专也怕属'

接下来采用相邻采样训练模型并创作歌词。

In [34]:
train_and_predict_rnn(rnn, get_rnn_params, init_rnn_hidden_state, 
                      num_hiddens, corpus_indices, vocab_size,
                      idx_to_char, char_to_idx, True, 
                      num_epochs, num_steps, lr, 
                      clipping_theta, batch_size, prefixes)

epoch 50, perplexity 1026.315377, time 17.07 sec
epoch 100, perplexity 1025.534982, time 17.64 sec
 - 分开 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 
 - 不分开 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 两 


## 4. `rnn`的简洁实现

PyTorch中的`nn`模块提供了循环神经网络的实现。下面构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层`rnn_layer`。

In [35]:
num_hiddens = 256
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)

与上一节中实现的循环神经网络不同，这里`rnn_layer`的输入形状为(时间步数, 批量大小, 输入个数)。其中输入个数即one-hot向量长度（词典大小）。此外，`rnn_layer`作为`nn.RNN`实例，在前向计算后会分别返回输出和隐藏状态h，其中输出指的是隐藏层在**各个时间步**上计算并输出的隐藏状态，它们通常作为后续输出层的输入。需要强调的是，该“输出”本身并不涉及输出层计算，形状为(时间步数, 批量大小, 隐藏单元个数)。而`nn.RNN`实例在前向计算返回的隐藏状态指的是隐藏层在**最后时间步**的隐藏状态：当隐藏层有多层时，每一层的隐藏状态都会记录在该变量中；对于像长短期记忆（LSTM），隐藏状态是一个元组(h, c)，即hidden state和cell state。我们会在本章的后面介绍长短期记忆和深度循环神经网络。

来看看我们的例子，输出形状为(时间步数, 批量大小, 隐藏单元个数)，隐藏状态h的形状为(层数, 批量大小, 隐藏单元个数)。

In [36]:
num_steps, batch_size, ini_state = 5, 2, None
X = torch.rand(num_steps, batch_size, vocab_size)  #  rnn_layer的输入: (时间步数, 批量大小, 输入神经元个数)
Y, state_new = rnn_layer(X, ini_state)
print(X.shape, Y.shape, len(state_new), state_new[0].shape)  # rnn_layer的输出: (时间步数, 批量大小, 输出神经元个数)

torch.Size([5, 2, 1027]) torch.Size([5, 2, 256]) 1 torch.Size([2, 256])


In [37]:
X

tensor([[[0.4477, 0.3952, 0.7353,  ..., 0.0715, 0.8597, 0.4895],
         [0.7692, 0.5618, 0.8165,  ..., 0.5675, 0.5608, 0.2502]],

        [[0.8439, 0.9982, 0.8990,  ..., 0.6277, 0.1539, 0.6503],
         [0.2940, 0.4328, 0.7230,  ..., 0.9115, 0.2356, 0.6854]],

        [[0.7304, 0.6079, 0.8652,  ..., 0.4536, 0.8823, 0.5138],
         [0.5567, 0.9643, 0.4606,  ..., 0.1508, 0.5373, 0.8820]],

        [[0.0643, 0.4040, 0.3013,  ..., 0.9756, 0.9160, 0.6333],
         [0.9729, 0.9785, 0.5211,  ..., 0.9541, 0.5276, 0.4733]],

        [[0.5521, 0.2243, 0.7783,  ..., 0.5306, 0.2259, 0.5449],
         [0.0169, 0.5206, 0.1594,  ..., 0.4572, 0.9048, 0.3838]]])

In [38]:
Y

tensor([[[-0.6532,  0.9053, -0.5501,  ...,  0.2196, -0.3451, -0.2091],
         [-0.1620,  0.7843, -0.5883,  ..., -0.0129, -0.2877, -0.2819]],

        [[-0.1960,  0.0520, -0.7968,  ...,  0.6025,  0.0737, -0.7011],
         [-0.3232,  0.2813, -0.8788,  ...,  0.3195,  0.5295, -0.8332]],

        [[-0.1283,  0.3974, -0.5997,  ...,  0.0765, -0.4186, -0.6938],
         [-0.3249,  0.2747, -0.6904,  ...,  0.2712,  0.1093, -0.8085]],

        [[ 0.4965,  0.4999, -0.7018,  ...,  0.6326,  0.2694, -0.0738],
         [-0.6393,  0.3907, -0.2546,  ...,  0.4868,  0.1042, -0.6694]],

        [[-0.8443,  0.3619, -0.6249,  ...,  0.1244, -0.0497, -0.0773],
         [-0.4401,  0.4278, -0.4027,  ...,  0.1721, -0.5707, -0.3785]]],
       grad_fn=<StackBackward>)

> 如果`rnn_layer`是`nn.LSTM`实例，那么上面的输出是什么？

### 4.1 定义模型

接下来我们继承`Module`类来定义一个完整的循环神经网络。它首先将输入数据使用one-hot向量表示后输入到`rnn_layer`中，然后使用全连接输出层得到输出。输出个数等于词典大小`vocab_size`。

In [39]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, num_hiddens):
        super(RNNModel, self).__init__()
        self.vocab_size = vocab_size  # 词典长度，即one-hot向量的长度
        self.rnn = nn.RNN(input_size=self.vocab_size, hidden_size=num_hiddens)  # 输入为one-hot向量，输出为隐藏层大小
        self.hidden_size = self.rnn.hidden_size * (2 if self.rnn.bidirectional else 1) 
        self.linear = nn.Linear(self.hidden_size, vocab_size)  # 输出为一个one-hot向量
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len), 输入的索引：批量，批量各时间步对应one-hot向量
        # 获取one-hot向量表示
        X = to_onehot(inputs, self.vocab_size) # X是1个list, (step, batch_size, num_neuros)
        Y, self.state = self.rnn(torch.stack(X), state)  # 批操作, 将X堆叠成1个批次
        # 全连接层首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.linear(Y.view(-1, Y.shape[-1]))
        return output, self.state

### 4.2 训练模型

同上一节一样，下面定义一个预测函数。这里的实现区别在于前向计算和初始化隐藏状态的函数接口。

In [40]:
def predict_rnn_concise(prefix, num_chars, model, vocab_size, idx_to_char, char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]]).view(1, 1)
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0], state[1])
            else:   
                state = state.to(device)
            
        Y, state = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

让我们使用权重为随机值的模型来预测一次。

In [41]:
vocab_size, num_hiddens

(1027, 256)

In [43]:
model = RNNModel(vocab_size, num_hiddens)
predict_rnn_concise('分开', 10, model, vocab_size, idx_to_char, char_to_idx)

'分开丁年么年么年戏年么年'

接下来实现训练函数。算法同上一节的一样，但这里只使用了相邻采样来读取数据。

In [47]:
def train_and_predict_rnn_concise(model, num_hiddens, vocab_size,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()  # 交叉熵损失函数
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # 优化算法
    state = None  # 初始状态
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.perf_counter()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps) # 相邻采样 data_iter_consecutive(corpus_indices, batch_size, num_steps)
        for X, Y in data_iter:  # X, Y 分别为1个批次对应的词下标
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance(state, tuple): # LSTM, state:(h, c)
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()
                    
            output, state = model(X.long(), state) # output: 形状为(num_steps * batch_size, vocab_size), state: batch_size*num_hidden
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # num_steps * batch 的向量，这样跟输出的行一一对应
            y = Y.t().contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            grad_clipping(model.parameters(), clipping_theta)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = np.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_concise(
                    prefix, pred_len, model, vocab_size, idx_to_char,
                    char_to_idx))

In [45]:
len(corpus_indices), batch_size, num_steps, device

(10000, 2, 5, device(type='cpu'))

使用和上一节实验中一样的超参数（除了学习率）来训练模型。

In [48]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_concise(model, num_hiddens, vocab_size,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 1.045098, time 1616561832.20 sec
 - 分开 我的世界已狂风暴雨 Wu  爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 
 - 不分开 我说 小去大来一记 仁者无敌 是谁在练太极 风生水起 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈
epoch 100, perplexity 1.045918, time 1616561832.19 sec
 - 分开的我面是你 想你想的都会笑 没有你在 我有多难熬  没有你在我有多难熬多烦恼  没有你烦 我有多烦恼
 - 不分开不休 我想一定是我听错弄错搞错 拜托 我想是你的脑袋有问题 随便说说 其实我早已经猜透看透不想多说 
epoch 150, perplexity 1.029264, time 1616561832.19 sec
 - 分开的我已无能为力的在人海中漂泊心伤透 娘子她人在江南等我 泪不休 语沉默娘子 娘子却依旧每日 折一枝杨
 - 不分开不天 你在空痛  其是我的爸爸一直打我妈妈 我说你的黑色幽默娘子 娘子却依旧每日 折一枝杨柳 你在那
epoch 200, perplexity 1.058134, time 1616561832.19 sec
 - 分开 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样
 - 不分开不再受宠 我的天空 是雨是风 还是彩虹 你在操纵 恨自己真的没用 情绪激动 一颗心到现在还在抽痛 还
epoch 250, perplexity 1.037471, time 1616561832.19 sec
 - 分开 我想就这样牵着你的手不放开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样
 - 不分开不了口让她知道 就一那么简单单爱的人睡着  想来我的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味


## 小结

* PyTorch的`nn`模块提供了循环神经网络层的实现。
* PyTorch的`nn.RNN`实例在前向计算后会分别返回输出和隐藏状态。该前向计算并不涉及输出层计算。

## 5. LSTM

In [ ]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

In [ ]:
def get_lstm_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

In [ ]:
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))

In [ ]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

简洁实现

In [ ]:
lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)

In [ ]:
lstm_layer

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, num_hiddens):
        super(RNNModel, self).__init__()
        self.vocab_size = vocab_size
        self.lstm = nn.LSTM(input_size=self.vocab_size, hidden_size=num_hiddens)
        self.hidden_size = self.lstm.hidden_size * (2 if self.rnn.bidirectional else 1) 
        self.linear = nn.Linear(self.hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = to_onehot(inputs, self.vocab_size) # X是个list
        Y, self.state = self.lstm(torch.stack(X), state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.linear(Y.view(-1, Y.shape[-1]))
        return output, self.state

In [ ]:
model = LSTMModel(vocab_size, num_hiddens)

## 6. GRU

In [ ]:
def get_gru_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

In [ ]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

简洁实现

In [ ]:
lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, num_hiddens):
        super(GRUModel, self).__init__()
        self.vocab_size = vocab_size
        self.gru = nn.GRU(input_size=self.vocab_size, hidden_size=num_hiddens)
        self.hidden_size = self.gru.hidden_size * (2 if self.gru.bidirectional else 1) 
        self.linear = nn.Linear(self.hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = to_onehot(inputs, self.vocab_size) # X是个list
        Y, self.state = self.gru(torch.stack(X), state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.linear(Y.view(-1, Y.shape[-1]))
        return output, self.state

In [ ]:
model = GRUModel(vocab_size, num_hiddens)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

## 附. NLPM

In [ ]:
import torch.nn.functional as F

In [ ]:
class NGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, hidden_size):
        super(NGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)  # 嵌入层
        self.linear1 = nn.Linear(context_size*embedding_dim, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inputs):
        # 将输入的单词编码映射为one-hot编码，然后经过一个线性层得到单词的词向量
        # inputs: 1*context_size
        embeds = self.embeddings(inputs) # -> context_size, vocab_size
        embeds = embeds.view(1, -1)  # -> 1, context_size*embedding_dim
        out = self.linear1(embeds)  # -> hidden_size
        out = F.relu(out)
        out = self.linear2(out)  # -> vocab_size
        return out
    
    def extract(self, inputs):
        embeds = self.embeddings(inputs)
        return embeds

In [ ]:
inputs = torch.LongTensor([1, 2, 4, 2])
vocab_size = 100
embedding_dim = 20
context_size = 3
hidden_size = 10
ngram = NGram(vocab_size, embedding_dim, context_size, hidden_size)

In [ ]:
ngram.extract(inputs)

In [ ]:
class CBOW(nn.Module):
    def __init__(self. vocab_size, embedding_dim, context_size, hidden_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size*embedding_dim, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = embeds.view(embeds.shape[0], -1)
        out = self.linear1(embeds)
        out = F.relu(out)
        out = self.linear2(out)
        return out
    
    def extract(self, inputs):
        embeds = self.embeddings(inputs)
        return embeds